# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [96]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

- в стобцах days_employed и total_income имеються пропуски
- в столбце education значения разного регистра
- в стобце days_employed значения отрицательные

### Шаг 2. Предобработка данных

In [97]:
# сделаю отдельный дата для каждого типа занятости, чтобы не ошибиться посмотрим какие типы занятости есть?

# print(df['income_type'].unique())

#['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный' 'предприниматель' 'студент' 'в декрете']

df_sotrudnik = df[(df['income_type'] == 'сотрудник')]
df_penciober = df[(df['income_type'] == 'пенсионер')]
df_companion = df[(df['income_type'] == 'компаньон')]
df_gosslujaci = df[(df['income_type'] == 'госслужащий')]
df_unemployed = df[(df['income_type'] == 'безработный')]
df_bisnesmen = df[(df['income_type'] == 'предприниматель')]
df_student = df[(df['income_type'] == 'студент')]
df_dikret = df[(df['income_type'] == 'в декрете')]

#print(df_sotrudnik.info())
#print(df_penciober.info())
#print(df_companion.info())
#print(df_gosslujaci.info())
#print(df_unemployed.info())
#print(df_bisnesmen.info())
#print(df_student.info())
#print(df_dikret.info())

# среди сотрудников 1105 из 11119 человек без стажа и дохода, среди пенсионеров 413 из 3856
# среди компаньонов 508 из 5085, среди госслужащих 147 из 1459, из предпринимателей 1 из 2-х


# буду полагать что пропуски появились по причине того что люди либо только устроились на работу, 
# либо взяли кредит под залог имущества, или же не успели принести справку в любом случае у них нет стажа и дохода, 
# поэтому заполним 0
# заполняю целиком таблицу, так как пропущенные значения только в столбцах ['days_employed'] ['total_income']
df = df.fillna(0) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

- Пропущенных значения заменил на 0, так как люди либо только устроились на работу, либо взяли кредит под залог имущества, или же не успели принести справку в любом случае у них не указан стаж и доход.

### Замена типа данных

In [98]:
# замените вещественный тип данных на целочисленный.
# переведем тип данных из float в int в столбцах: ['days_employed'] и ['total_income']

df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')
df.info()

# df.head()

# рассмотрим графу ['days_employed'], в ней есть отрицательные и положительные значения, стаж у нас не бывает отрицательным.

# df_plus = df[df['days_employed'] > 0]
# df_minus = df[df['days_employed'] < 0]

#просмотрим минимальные и максимальные значения в каждом случае

# print(df_plus['days_employed'].min(), df_plus['days_employed'].max())
# print(df_minus['days_employed'].min(), df_minus['days_employed'].max())

# трудовой стаж исчисляется в днях, в связи с этим в положительных данных ошибка.

# print(df_plus['days_employed'].count())
# print(df_minus['days_employed'].count())

# положительных данных намного меньше, поэтому ими мы можем пожертвовать, чтобы можно было использовать более понятную информацию

df = df[df['days_employed'] <= 0]
df['days_employed'] = df['days_employed'] * -1

# print(df.head())
# print(df['days_employed'].min(), df['days_employed'].max())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


### Вывод

- Теперь в графе ['days_employed'] со стажем только правельные показатели.

### Обработка дубликатов

In [99]:
# определим где и какие дубликаты есть
# приведем все данные типа object к нижнему регистру для дальнейшей простоты обработки данных
df['education'] = df['education'].str.lower()
df['family_status'] = df['family_status'].str.lower()
df['gender'] = df['gender'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['purpose'] = df['purpose'].str.lower()
# Все данные типа object приведены к нижнему регистру для простоты обработки данных и упрощения дальнейшего поиска дубликатов.



# Посмотрим общее количество заемщиков с разным уровнем образования.
df.sort_values('education')['education'].value_counts()

# Удалим те же строки в массиве данных.
df = df.drop_duplicates().reset_index(drop = True)

# Теперь посмотрим общее количество дубликатов
df.duplicated(keep=False).reset_index(drop=True).sum()




print('Уникальные значения в графе education:', df['education'].unique())
print()
print('Уникальные значения в графе family_status:', df['family_status'].unique())
print()
print('Уникальные значения в графе gender:', df['gender'].unique())
print()
print('Уникальные значения в графе income_type:', df['income_type'].unique())
print()
print('Уникальные значения в графе purpose:', df['purpose'].unique())
print()
# оказывается в колонке 'gender' есть значения 'xna' (не изветсен пол) в колонке 'purpose' много взаимозаменяемых значений, 
# например 'приобретение автомобиля' и 'автомобиль'

Уникальные значения в графе education: ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']

Уникальные значения в графе family_status: ['женат / замужем' 'гражданский брак' 'в разводе' 'не женат / не замужем'
 'вдовец / вдова']

Уникальные значения в графе gender: ['f' 'm' 'xna']

Уникальные значения в графе income_type: ['сотрудник' 'компаньон' 'пенсионер' 'госслужащий' 'предприниматель'
 'студент' 'в декрете']

Уникальные значения в графе purpose: ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'операции с жильем' 'образование' 'на проведение свадьбы'
 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'сыграть свадьбу'
 'покупка жилой недвижимости' 'строительство собственной недвижимости'
 'недвижимость' 'строительство недвижимости'
 'на покупку своего автомобиля' 'на покупку подержанного автомобиля'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться обр

- Преобразованные значения в столбце ['education'] к одиному регистру. Используя метод .duplicated(), мы рассчитали количество дубликатов, строк с повторяющимися значениями в индексах. Убрали все те же строки.
- Оказывается в колонке ['gender'] есть значения 'xna' (не изветсен пол)
в колонке ['purpose'] много взаимозаменяемых значений, например 'приобретение автомобиля' и 'автомобиль'

### Лемматизация

In [100]:
# для добавления создания лемм
from pymystem3 import Mystem
m = Mystem()

def do_lemma(row):
    lemma = m.lemmatize(row)
    return lemma

# создадим дополнительный столбец с лемматизированными целями
df['purpose_lemma'] = df['purpose'].apply(do_lemma)

# создадим строку из уникальных целей
all_lemmas_list = df['purpose_lemma'].values
flat_lemmas_list = []
for sublist in all_lemmas_list:
   for item in sublist:
       flat_lemmas_list.append(item)
        
print(all_lemmas_list)
from collections import Counter

# посчитаем самые популярные слова, чтобы обобщить цели кредитов, это: недвижимость, образование, автомобиль, свадьба
print('Посчитаем самые популярные леммы в графе purpose:', Counter(flat_lemmas_list))

# функция для создания словаря
def dictionary(world, data):
    for row in data:
        if world == row:
            return row

# создадим словарь с категориями, вдальнейшем его можно будет пополнять

dict = [] # вручную добавим выбранные мною слова
dict.append(dictionary('жилье', flat_lemmas_list))
dict.append(dictionary('недвижимость', flat_lemmas_list))
dict.append(dictionary('образование', flat_lemmas_list))
dict.append(dictionary('автомобиль', flat_lemmas_list))
dict.append(dictionary('свадьба', flat_lemmas_list))

print('Cоздадим словарь с категориями в графе purpose, вдальнейшем его можно будет пополнять:', dict)

[list(['покупка', ' ', 'жилье', '\n'])
 list(['приобретение', ' ', 'автомобиль', '\n'])
 list(['покупка', ' ', 'жилье', '\n']) ... list(['недвижимость', '\n'])
 list(['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n'])
 list(['на', ' ', 'покупка', ' ', 'автомобиль', '\n'])]
Посчитаем самые популярные леммы в графе purpose: Counter({' ': 28191, '\n': 18009, 'недвижимость': 5341, 'покупка': 4964, 'жилье': 3768, 'автомобиль': 3591, 'образование': 3359, 'с': 2449, 'операция': 2191, 'свадьба': 1950, 'свой': 1907, 'на': 1857, 'строительство': 1568, 'высокий': 1160, 'коммерческий': 1099, 'получение': 1092, 'для': 1080, 'жилой': 1033, 'сделка': 788, 'заниматься': 769, 'дополнительный': 744, 'проведение': 642, 'сыграть': 638, 'сдача': 549, 'собственный': 533, 'семья': 531, 'со': 530, 'ремонт': 516, 'подержанный': 407, 'подержать': 394, 'приобретение': 384, 'профильный': 366})
Cоздадим словарь с категориями в графе purpose, вдальнейшем его можно будет пополнять: ['жилье', 'недвижимость'

### Вывод

### Категоризация данных

In [101]:
# создаем столбец с категориями
# функция которая выдает категории
def category(data):
    if dict[0] in data:
        return dict[0]
    elif dict[1] in data:
        return dict[1]
    elif dict[2] in data:
        return dict[2]
    elif dict[3] in data:
        return dict[3]
    else:
        return dict[4]
    
# Cоздаем столбец с категориями целей получения кредита
df['category_purpose'] = df['purpose_lemma'].apply(category)
print(df.loc[:, ['purpose', 'category_purpose']].head(10))

# функция которая выдает категории дохода
# Разобьем доход на уровни: 0, 0-20, 20-50, 50-100, 100-250, 250-500, 500
# Выбрал такое деление на доходность для более детального среза информации.

def income(data):
    if data == 0:
        return '0'
    elif data <= 20000:
        return '0-20'
    elif data <= 50000:
        return '20-50'
    elif data <= 100000:
        return '50-100'
    elif data <= 250000:
        return '100-250'
    elif data <= 500000:
        return '250-500'
    else:
        return '500+'

df['category_income'] = df['total_income'].apply(income)

# Cоздаем столбец с категориями
print(df.loc[:, ['total_income', 'category_income']] .head(10))

                      purpose category_purpose
0               покупка жилья            жилье
1     приобретение автомобиля       автомобиль
2               покупка жилья            жилье
3  дополнительное образование      образование
4               покупка жилья            жилье
5           операции с жильем            жилье
6                 образование      образование
7       на проведение свадьбы          свадьба
8     покупка жилья для семьи            жилье
9        покупка недвижимости     недвижимость
   total_income category_income
0        253875         250-500
1        112080         100-250
2        145885         100-250
3        267628         250-500
4        255763         250-500
5        240525         100-250
6        135823         100-250
7         95856          50-100
8        144425         100-250
9        113943         100-250


### Вывод

- cоздал столбец с категориями целей получения кредита
+ cоздал столбец с категориями доходности

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [102]:
df_kindred = df[df['children'] != 0]
df_freekind = df[df['children'] == 0]

kindred_debit = df_kindred['debt'].sum() / len(df_kindred)
freekind_debit = df_freekind['debt'].sum() / len(df_freekind)

print('Процент среди людей с детьми, которые имели задолжности: {:.1%}'.format(kindred_debit))
print('Процент среди бездетных, которые имели задолжности: {:.1%}'.format(freekind_debit))

Процент среди людей с детьми, которые имели задолжности: 9.4%
Процент среди бездетных, которые имели задолжности: 8.2%


### Вывод

Казалось бы что люди, которые заводят детей более ответственно относятся к выплате долгов, но на деле оказалось наоборот.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [103]:
# Типы семейного положения и их id соответственно:
print(df['family_status'].unique())
print(df['family_status_id'].unique())
# ['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе' 'не женат / не замужем']
# [0 1 2 3 4]

# создаем таблицу data_pivot_family_status где смотрим дожников по категории семейного положения
data_pivot_family_status = df.pivot_table(index='family_status', columns='debt', values='family_status_id', aggfunc='count')
# считаем процент должников
data_pivot_family_status['ratio %'] = (data_pivot_family_status[1] / data_pivot_family_status[0]) * 100
data_pivot_family_status['ratio %'] = data_pivot_family_status['ratio %'].astype('int')
# cчитаем сумму клиентов каждой категории
data_pivot_family_status['sum'] = data_pivot_family_status[1] + data_pivot_family_status[0]


print(data_pivot_family_status.loc[:, ['ratio %', 'sum']])

['женат / замужем' 'гражданский брак' 'в разводе' 'не женат / не замужем'
 'вдовец / вдова']
[0 1 3 4 2]
debt                   ratio %    sum
family_status                        
в разводе                    8    997
вдовец / вдова               6    475
гражданский брак            11   3574
женат / замужем              8  10466
не женат / не замужем       11   2497


### Вывод

Из таблицы видно что те люди кто никогда не был в официальном браке более склонны к задолжностям: Гражданский брак, не женат / не замужем - 10% с задолжностями. В разводе, вдовец / вдова, женат / замужем - 7-8% с задолжностями.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [104]:
# Разобьем доход на уровни: 0, 0-20, 20-50, 50-100, 100-250, 250-500, 500
# Выбрал такое деление на доходность для более детального среза информации.
#создаем таблицу data_pivot_income где смотрим дожников по категории заработка
data_pivot_income = df.pivot_table(index='category_income', columns='debt', values='total_income', aggfunc='count')
#считаем процент должников
data_pivot_income['ratio %'] = (data_pivot_income[1] / data_pivot_income[0]) * 100
data_pivot_income['ratio %'] = data_pivot_income['ratio %'].astype('int')
#cчитаем сумму клиентов каждой категории
data_pivot_income['sum'] = data_pivot_income[1] + data_pivot_income[0]

print(data_pivot_income.loc[:, ['ratio %', 'sum']])

debt             ratio %    sum
category_income                
0                      8   2103
100-250                9  10199
20-50                  7    192
250-500                7   2328
50-100                10   2984
500+                   7    203


Из таблицы видно что те люди кто имеет заработок выше среднего более склонны к задолжностям: 50-250 тысяч рублей в месяц - 9-10% с задолжностями до 50 и свыше 250 тысяч рублей в месяц - 7% с задолжностями.

- Как разные цели кредита влияют на его возврат в срок?

In [105]:
#создаем таблицу data_pivot_income где смотрим дожников по цели кредита
data_pivot_purpose = df.pivot_table(index='category_purpose', columns='debt', values='purpose', aggfunc='count')
#считаем процент должников
data_pivot_purpose['ratio %'] = (data_pivot_purpose[1] / data_pivot_purpose[0]) * 100
data_pivot_purpose['ratio %'] = data_pivot_purpose['ratio %'].astype('int')
#cчитаем сумму клиентов каждой категории
data_pivot_purpose['sum'] = data_pivot_purpose[1] + data_pivot_purpose[0]

print(data_pivot_purpose.loc[:, ['ratio %', 'sum']])

debt              ratio %   sum
category_purpose               
автомобиль             11  3591
жилье                   7  3768
недвижимость            8  5341
образование            10  3359
свадьба                 9  1950


### Вывод

Из таблицы видно что те люди кто берет кредит на более мелкие цели более склонны к задолжностям: Автомобиль, образование, свадьба - 9-11% с задолжностями: Жилье, недвижимость - 7-8% с задолжностями.

### Шаг 4. Общий вывод

Из выше указанных исследований видно что более надежными оказались люди, которые были или состоят в браке, имеют ЗП, до 50 и свыше 250 тысяч рублей в месяц и с более масштабной целью кредита, например жилье или недвижимость.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.